##  Sentinel-3 SLSTR composites

### Service Definition

In [1]:
service = dict([('title', 'Sentinel-3 SLSTR Level-2 reprojection and tiling'),
                ('abstract', 'This service takes as input a Sentinel-3 SLSTR Level 2 (SL_2_LST____) product on DESCENDING pass and does the reprojection and tiling'),
                ('id', 'ewf-wfp-03-01-02')])

### Parameter Definition 

### Runtime parameter definition

**Input reference**

The input identifier is the catalogue entry URL (a.k.a. self value).

In [2]:
input_reference = dict([('identifier', 'input_reference'),
                        ('title', 'Sentinel-3 SLSTR Level-2 (SL_2_LST____ descending pass)'),
                        ('abstract', 'This service takes as input a Sentinel-3 SLSTR Level 2 (SL_2_LST____) product on DESCENDING pass'),
                        ('value', 'https://catalog.terradue.com/sentinel3/search?format=json&uid=S3B_SL_2_LST____20200613T070835_20200613T071135_20200613T085802_0179_040_063_2880_LN2_O_NR_004'),
                        ('stac:collection', 'input_reference'),
                        ('stac:href', 'catalog.json'),
                        ('max_occurs', '16')])

In [3]:
tiling_level = dict([('identifier', 'tiling_level'),
                ('value', '5'),
                ('title', 'Tiling level'),
                ('abstract', 'Tiling level'),
                ('max_occurs', '1')])


**Data path**

This path defines where the data is staged-in. 

In [4]:
data_path = '/workspace/data/slstr'

In [5]:
input_catalog = '/workspace/data/slstr/catalog.json'

### Workflow

#### Import the packages

In [6]:
import os
os.environ['PREFIX'] = '/opt/anaconda/envs/env_s3/'
import sys
sys.path.append('.')
import gdal
from helpers import *
from shapely.wkt import loads
from shapely.geometry import box
from shapely.geometry import shape
import shutil
from pystac import Catalog, Collection, Item, MediaType, Asset, CatalogType

from tiling import s3_tiles
import time
import glob

gdal.UseExceptions()

ImportError: /opt/anaconda/envs/env_s3/lib/python3.5/site-packages/osgeo/../../../libgdal.so.20: undefined symbol: _ZN6kealib10KEAImageIO16setImageMetaDataENSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEES6_

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
cat = Catalog.from_file(input_catalog)

if cat is None:
    raise ValueError()

In [ ]:
collection = next(cat.get_children())

In [ ]:
item = next(collection.get_items())

In [ ]:
item.properties['eop:orbitDirection']

In [ ]:
if item.properties['eop:orbitDirection'] != 'DESCENDING':
    ciop.log('ERROR','Product cannot be used as input')
    raise Exception('Use products with descending orbit direction')

In [ ]:
s3_wkt = shape(item.geometry).wkt

### Import Sentinel-3 SLSTR product

In [ ]:
operators = ['Read', 
             'Reproject',
             'Write']

In [ ]:
read = dict()

s3_path = item.assets['metadata'].get_absolute_href()

read['file'] =  s3_path
read['formatName'] = 'Sen3'

reproject = dict()
reproject['crs'] = 'EPSG:4326'

write = dict()
write['file'] = 's3_slstr'

In [ ]:
snap_graph(os.environ['GPT_BIN'],
           operators,
           Read=read, 
           Reproject=reproject,
           Write=write)

In [ ]:
def s3_to_tile(input_tif, item, tile):
    
    translate_options = gdal.TranslateOptions(gdal.ParseCommandLine("-co TILED=YES -co COPY_SRC_OVERVIEWS=YES -co COMPRESS=LZW"))
        
    x_min, y_min, x_max, y_max = tile.tile.bounds

    output_tile_name = '{}_L{}_C{}_R{}'.format(item.id,
                                                    tile.level,
                                                    tile.col,
                                                    tile.row)


    gdal.Translate('tmp_{}.tif'.format(output_tile_name),
                   input_tif,
                   projWin=[x_min, y_max, x_max, y_min],
                   projWinSRS='EPSG:4326')

    ds = gdal.Open('tmp_{}.tif'.format(output_tile_name),
                   gdal.OF_READONLY)

    gdal.SetConfigOption('COMPRESS_OVERVIEW', 'DEFLATE')
    ds.BuildOverviews('NEAREST', [2,4,8,16,32])
    ds = None

    ds = gdal.Open('tmp_{}.tif'.format(output_tile_name))
    ds = gdal.Translate('{}.tif'.format(output_tile_name), ds, options=translate_options)
    ds = None

    band_names = ['LST', 'NDVI', 'Land mask', 'Cloud mask']

    ds = gdal.Open('{}.tif'.format(output_tile_name), gdal.GA_Update)

    # update extended area to -10000 instead of 0 as gdal.trasnlate does
    ndvi_data = ds.GetRasterBand(1).ReadAsArray()
    land_data = ds.GetRasterBand(2).ReadAsArray()
    cloud_data = ds.GetRasterBand(3).ReadAsArray()
    
    updated_ndvi_data = np.full(ndvi_data.shape, -10000)
    
    updated_ndvi_data = np.where((land_data == 1) | (cloud_data == 1), 
                                 ndvi_data, -10000)
    
    ds.GetRasterBand(1).WriteArray(updated_ndvi_data)
    
    ds.FlushCache()
    
    ds = None
    
    ds = gdal.Open('{}.tif'.format(output_tile_name), gdal.GA_Update)
    
    for index in range(ds.RasterCount):

        srcband = ds.GetRasterBand(index+1)

        srcband.SetDescription(band_names[index])

        if index == 0:
            
            srcband.SetNoDataValue(-10000)
    
    ds.FlushCache()

    ds = None
    
    with open(output_tile_name + '.properties', 'w') as file:

        file.write('title=Tile L:{1} C:{2} R:{3} {0}\n'.format(item.id,
                                                              tile.level, 
                                                              tile.col, 
                                                              tile.row))
        
        file.write('date={}/{}\n'.format(item.datetime.strftime('%Y-%m-%dT%H:%M:%sZ'), 
                                         item.datetime.strftime('%Y-%m-%dT%H:%M:%sZ')))
        
        file.write('geometry={0}'.format(tile.tile.wkt))


    for f in ['tmp_{}.tif'.format(output_tile_name), 'tmp_{}.tif.ovr'.format(output_tile_name)]:

        if os.path.exists(f):

            os.remove(f)
    
    return True
    

### Tiling

In [ ]:
bands = [os.path.join('s3_slstr.data', '{}.img'.format(band)) for band in ['LST', 'NDVI', 'cloud_in', 'confidence_in']]
        
s3_data = read_s3(bands)
        

In [ ]:
ds = gdal.Open(bands[0])

geo_transform = ds.GetGeoTransform()
projection_ref = ds.GetProjectionRef()
    

In [ ]:
lst = s3_data[:,:,0]
ndvi = s3_data[:,:,1]
cloud = s3_data[:,:,2]
confidence = s3_data[:,:,3]

In [ ]:
mask = get_slstr_confidence_mask('land', confidence)

In [ ]:
cloud_mask =  get_slstr_mask('gross_cloud', cloud)

In [ ]:
output_name = 'temp.tif'

In [ ]:
driver = gdal.GetDriverByName('GTiff')

output = driver.Create(output_name, 
                       lst.shape[1], 
                       lst.shape[0], 
                       4, 
                       gdal.GDT_Float32)

output.SetGeoTransform(geo_transform)
output.SetProjection(projection_ref)
output.GetRasterBand(1).WriteArray(lst)
output.GetRasterBand(2).WriteArray(ndvi)
output.GetRasterBand(3).WriteArray(mask)
output.GetRasterBand(4).WriteArray(cloud_mask)

output.FlushCache()

output = None

In [ ]:
tiles = s3_tiles(shape(item.geometry), int(tiling_level['value']))


In [ ]:
tiles

In [ ]:
for index, tile in tiles.iterrows():
    
    logging.info('Tile L{} C{} R{}'.format(tile.level,
                                       tile.col,
                                       tile.row))


    s3_to_tile(output_name, item, tile)

In [ ]:
logging.info('Clean-up') 
os.remove(output_name)

shutil.rmtree('s3_slstr.data')
os.remove('s3_slstr.dim')

time.sleep(45)

for f in glob.glob('./*.tif.aux.xml'):

    os.remove(f)


### License

This work is licenced under a [Attribution-ShareAlike 4.0 International License (CC BY-SA 4.0)](http://creativecommons.org/licenses/by-sa/4.0/) 

YOU ARE FREE TO:

* Share - copy and redistribute the material in any medium or format.
* Adapt - remix, transform, and built upon the material for any purpose, even commercially.

UNDER THE FOLLOWING TERMS:

* Attribution - You must give appropriate credit, provide a link to the license, and indicate if changes were made. You may do so in any reasonable manner, but not in any way that suggests the licensor endorses you or your use.
* ShareAlike - If you remix, transform, or build upon the material, you must distribute your contributions under the same license as the original.